In [2]:
%pip install --upgrade --quiet  langchain langchain-community langchain-experimental langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
f = open('openaikey.txt')
os.environ["OPENAI_API_KEY"]=f.read()

In [3]:
# Examples of job roles and respective job titles
examples = [
    {"input": "software engineer", "output": "software development"},
    {"input": "accountant", "output": "accounting"},
    {"input": "teacher", "output": "education"},
    {"input": "doctor", "output": "medicine"},
    {"input": "architect", "output": "architecture"},
    {"input": "lawyer", "output": "law"},
]
# This is the list of examples available to select from.

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAIEmbeddings,ChatOpenAI

chatAI = ChatOpenAI()

# SemanticSimilarityExampleSelector will select examples that are similar to your input by semantic meaning
selector = SemanticSimilarityExampleSelector.from_examples(
    examples, # list of examples available to select from.
    OpenAIEmbeddings(), # embedding class used to produce embeddings which are used to measure semantic similarity.
    FAISS, # VectorStore class that is used to store the embeddings and do a similarity search over.
    k=2  # number of examples to produce.
)


In [5]:
from langchain_core.prompts import FewShotPromptTemplate
example_prompt = PromptTemplate.from_template("User input: {input}\nOutput: {output}")
similar_prompt = FewShotPromptTemplate(example_selector=selector, # The object that will help select examples
									   example_prompt=example_prompt,  # Your prompt
    								   prefix="Give the job title their job role is ", # Customizations that will be added to the top and bottom of your prompt
    								   suffix="Input: {job_title}\nOutput:",
    								   input_variables=["job_title"] # What inputs your prompt will receive
    								   )


In [10]:
prompt = similar_prompt.format(job_title="Cricketer")
print(prompt)

Give the job title their job role is 

User input: doctor
Output: medicine

User input: software engineer
Output: software development

Input: Cricketer
Output:


In [11]:
result = chatAI.invoke(prompt)

In [12]:
print(result)

content='professional sports player' response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 44, 'total_tokens': 47}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-933ad6d0-2a91-47e4-a4ca-51c21b7ad0f4-0' usage_metadata={'input_tokens': 44, 'output_tokens': 3, 'total_tokens': 47}
